# Maximum Number of Employees Reached

Write a query that returns every employee that has ever worked for the company. For each employee, calculate the greatest number of employees that worked for the company during their tenure and the first date that number was reached. The termination date of an employee should not be counted as a working day. 



Your output should have the employee ID, greatest number of employees that worked for the company during the employee's tenure, and first date that number was reached.

# Approaches


- Get those with hire_date & create column = 1, get those with termination_date & create column = -1 .. finally, concat those 2 dataframes

- CumSum columns

- Fetch earliest date where max employee was reached

- Order by id, maxemp, and date

In [1]:
# Import your libraries
import pandas as pd
import numpy as np
import datetime as dt
from getdata_fromhtml import getdata_fromhtml
# Start writing code
path = './data/uber_employee.html'

df = getdata_fromhtml(path) #uber_employees#.head()
df = df.replace(r'^\s*$', 'NaT', regex=True)

#start date of an employee 
start = df['hire_date'].to_frame('date')
start['value'] = 1 # get hired 
#start

end_date = df['termination_date'].to_frame('date').dropna()
end_date['value'] = -1
#end_date
# Concatinate the dates 
dates = pd.concat([start, end_date], ignore_index = True)

#count the number of employess for each day 
dates = dates.groupby('date')['value'].sum().reset_index()

# Order the dataset in ascending order according to the date 
dates = dates.sort_values('date')
dates['cum_emp_count'] = dates['value'].cumsum()

# Find the 
def max_employe_min_date(x):
    if df.at[x, 'termination_date'] is pd.NaT:
        end_date = dt.datetime.today()
    else:
        end_date = df.at[x, 'termination_date']
    startdate = df.at[x, 'hire_date']
    max_no_emp = dates[dates['date'].between(startdate, end_date)]['cum_emp_count'].max()
    df.at[x, 'max_cont'] = max_no_emp
    # Find the earliest date that the max_no_emp is obtained 
    early_date = dates[(dates['cum_emp_count'] == max_no_emp) & (dates['date'].between(startdate, end_date))]['date'].min()
    df.at[x, 'earliest_date'] = early_date
    return df
    
for i in range(0, df.shape[0]):
    max_employe_min_date(i)
#lis = list(range(0, df.shape[0]))
#df_new = pd.DataFrame(map(max_employe_min_date, lis))
#print(df_new)
#df_new = df_new[['id', 'max_cont', 'earliest_date']]
result = df[['id', 'max_cont', 'earliest_date']] 
    
result.head()

,id,max_cont,earliest_date
0,1,57.0,2015-04-10
1,2,58.0,2017-02-03
2,3,58.0,2017-02-03
3,4,48.0,2013-12-23
4,5,58.0,2017-02-03


# Distance Per Dollar
- Hard

You’re given a dataset of uber rides with the traveling distance (‘distance_to_travel’) and cost (‘monetary_cost’) for each ride. For each date, find the difference between the distance-per-dollar for that date and the average distance-per-dollar for that year-month. Distance-per-dollar is defined as the distance traveled divided by the cost of the ride.



The output should include the year-month (YYYY-MM) and the absolute average difference in distance-per-dollar (Absolute value to be rounded to the 2nd decimal). 

You should also count both success and failed request_status as the distance and cost values are populated for all ride requests. Also, assume that all dates are unique in the dataset. Order your results by earliest request date first.

# Approaches

- Find the distance-per-dollar by dividing the distance and cost columns 

- Add a month column extracted from the request_date column to help with grouping the distance-per-dollar by month 

- Use a window function to find the average monthly distance-per-dollar for each month

- Calculate the absolute difference between the distance_to_cost for each request_date and month



In [2]:
# Import your libraries
import pandas as pd
import datetime as dt

path = './data/uber_requests_logs.html'
# Start writing code
df = getdata_fromhtml(path)
cnts = df[df['request_status'] == 'success'].shape[0]
cntf = df[df['request_status'] == 'fail'].shape[0]

df['request_date'] = df['request_date'].apply(pd.to_datetime)
df['year_month'] =  df['request_date'].dt.to_period('M')
df['dis_per_dollar'] = (df['distance_to_travel'].astype(float))/df['monetary_cost'].astype(float)

df['month_avg'] =  df.groupby(['year_month'])['dis_per_dollar'].transform('mean')#.reset_index()

#diff = round(diff, 2)

df['diff'] = abs(df['dis_per_dollar'] - df['month_avg'])
 
resu = df.groupby('year_month')['diff'].mean().reset_index()
resu['diff'] = round(resu['diff'], 2)
resu.head()

,year_month,diff
0,2020-01,3.32
1,2020-02,1.58
2,2020-03,3.13
3,2020-04,1.19
4,2020-05,6.44


--------------------------------------
# Highest Salary In Department
- Medium (twitter)

Find the employee with the highest salary per department. Output the department name, employee's first name along with the corresponding salary.

# Approach
- Using the Indexing method that is df[df[column]== condition] to filter out specific row values, specify the condition being the highest salary per department by using groupby() then applying transform(max) to look for this values per group in the dataframe

- Use [ [ column_name/s] ] to return a specified column of the dataframe


In [3]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/employee.html')
df = df[df.groupby(['department'])['salary'].transform('max')==df['salary']]
resu = df[['first_name', 'department', 'salary']]  #transform('max')
resu.head()

,first_name,department,salary
2,Richerd,Management,250000
18,Mick,Sales,2200
22,Michale,Audit,700


-----------------
# New Products

You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 for the first time with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year. If a company is new or had no products in 2019, then any product released in 2020 would be considered as new.

# Approach 

- Create 2 data frames where you're filtering the table for launches by 2020 and 2019

- Calculate distinct model launches in 2020 with 2029 using groupby and nunique and save the output in new data frames respectively

- OUTER JOIN 2020 grouped data frame with 2019 data frame

- Derive a new variable with difference in distinct launch count in 2 consecutive years

- Filter out all the company name where this difference greater than 0

- Output should be company name sorted in ascending order

In [4]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/car_launches.html')
df['year'] = df['year'].apply(pd.to_numeric)

df_19 = df[df['year'] == 2019].add_prefix('2019_')
df_19 = df_19.groupby('2019_company_name')['2019_product_name'].nunique().to_frame('nunique_prod_19').reset_index()

df_20 = df[df['year'] == 2020].add_prefix('2020_')
df_20 = df_20.groupby('2020_company_name')['2020_product_name'].nunique().to_frame('nunique_prod_20')

merged = pd.merge(df_19, df_20, left_on = '2019_company_name', right_on = '2020_company_name', how= 'outer')
merged['diff'] = merged['nunique_prod_20'] - merged['nunique_prod_19'] 
#merged = merged[merged['diff'] >= 0]
res = merged[['2019_company_name', 'diff']]
res.sort_values('2019_company_name')

,2019_company_name,diff
0,Chevrolet,2
1,Ford,-1
2,Honda,-3
3,Jeep,1
4,Toyota,-1


---------------------------
# Naive Forecasting
- Hard (Uber)

Some forecasting methods are extremely simple and surprisingly effective. Naïve forecast is one of them; we simply set all forecasts to be the value of the last observation. Our goal is to develop a naïve forecast for a new metric called "distance per dollar" defined as the (distance_to_travel/monetary_cost) in our dataset and measure its accuracy.



To develop this forecast,  sum "distance to travel"  and "monetary cost" values at a monthly level before calculating "distance per dollar". This value becomes your actual value for the current month. The next step is to populate the forecasted value for each month. This can be achieved simply by getting the previous month's value in a separate column. Now, we have actual and forecasted values. This is your naïve forecast. Let’s evaluate our model by calculating an error matrix called root mean squared error (RMSE). RMSE is defined as sqrt(mean(square(actual - forecast)). Report out the RMSE rounded to the 2nd decimal spot.

In [5]:
from getdata_fromhtml import getdata_fromhtml as g
df = g('./data/uber_requests_logs.html')

import pandas as pd
import datetime as dt
import numpy as np

df['request_date'] = df['request_date'].apply(pd.to_datetime)
df[['distance_to_travel', 'monetary_cost']] = df[['distance_to_travel', 'monetary_cost']].apply(pd.to_numeric)
df['year_month'] = df['request_date'].dt.to_period('M')
df = df.groupby('year_month').agg({'distance_to_travel': 'sum', 'monetary_cost': 'sum'}).reset_index()
df['distance_per_dollar'] = df['distance_to_travel']/df['monetary_cost']
df['prev_dist_per_dollar'] = df['distance_per_dollar'].shift()


rmse = (np.mean((df.iloc[1:, 3] - df.iloc[1:, 4])**2))**0.5 
round(rmse, 2)

2.34

-----------------------
# Year Over Year Churn
- Hard (lyft)

Find how the number of drivers that have churned changed in each year compared to the previous one. Output the year (specifically, you can use the year the driver left Lyft) along with the corresponding number of churns in that year, the number of churns in the previous year, and an indication on whether the number has been increased (output the value 'increase'), decreased (output the value 'decrease') or stayed the same (output the value 'no change').

# Approach
- Clean the end_date column by coverting it to datetime by applying pd.to_datetime

- Create a column that contains the count of those churned entries. Do this by making a Boolean condition where end_date is notnull() then converting it to numeric using astype(int)

- Create a column that contains the year under the end_date column using dt.year

- Group by year end using groupby() and get the sum of those who churned using sum(); convert the resulting object to a dataframe using to_frame('column_name').reset_index()

- Use shift(n) to offset alignment by n rows; replace the null values with 0

- Calculate the percentage variation as 100*(actual_number_churns - prev_year_number_churns)/prev_year_number_churns.


In [6]:
from getdata_fromhtml import getdata_fromhtml as g
import pandas as pd
import datetime as dt

df = g('./data/lyft_drivers.html')
  
df['end_date'] = pd.to_datetime(df['end_date'])
df =df[df['end_date'].notnull()]
df['year'] = df['end_date'].dt.year
df['count'] = 1
df = df.groupby('year')['count'].sum().to_frame('n_churned').reset_index().sort_values('year')

df['prev_chu'] = df['n_churned'].shift(1)

df = df.fillna(0.0)

#f['rate'] = 100*(df['n_churned'] - df['prev_chu'])/df['n_churned']
#
def comparison(x):
    if x < 0:
        return 'decrease'
    elif x > 0:
        return 'increase'
    else: return 'no change'

df['diff'] = df['n_churned'] - df['prev_chu']

df['measure'] = df['diff'].apply(comparison)
res = df[['year', 'n_churned', 'prev_chu', 'measure']]
res.head()

,year,n_churned,prev_chu,measure
0,2015,5,0.0,increase
1,2016,5,5.0,no change
2,2017,8,5.0,increase
3,2018,25,8.0,increase
4,2019,7,25.0,decrease


# Distances Traveled

- Medium

Find the top 10 users that have traveled the greatest distance. Output their id, name and a total distance traveled.

# Appoach

- Merge lyft_users and lyft_rides_log tables to get all the information using an inner merge

- Group by user_id to count total distance traveled by each user

- Sum distances per user to get total traveled distance for each user

- Sort the Dataframe by distance in descending order and select only top 10 rows by distance travelled

- Return a dataframe with 2 columns (user_id, distance)


In [7]:
import pandas as pd
from getdata_fromhtml import getdata_fromhtml as g
lyft_users = g('./data/lyft_users.html')
lyft_rides_log = g('./data/lyft_rides_log.html')

merged = pd.merge(lyft_users, lyft_rides_log, left_on ='id', right_on = 'user_id', how = 'inner')
df = merged.groupby(['user_id', 'name'])['distance'].sum().reset_index()
df = df.sort_values('distance', ascending = False)
df.iloc[0:10, :].head()

,user_id,name,distance
16,26,Barbara Larson,9894
40,7,Christopher Schmitt,96
36,48,Pamela Cox,942958
41,8,Patrick Gutierrez,93477429
38,5,Sean Parker,92
